In [1]:
# Importations
import pandas as pd
import numpy as np
from collections import Counter 
from sklearn.feature_extraction import DictVectorizer
import pickle
from sklearn.metrics import roc_auc_score, f1_score, log_loss, make_scorer
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from joblib import dump, load
# importting all dataset 

pd.set_option('display.max_columns', None)
import warnings; warnings. simplefilter('ignore')

In [2]:
train =  pd.read_csv("train.csv")
train_label = pd.read_csv("trainLabels.csv", engine='c')
train_label.drop(train_label.index[9999:49999],0,inplace=True)


In [3]:
# now merging two files in one data set
train_with_labels = pd.merge(train, train_label, on='id')


In [4]:
# Now importing testing dataset
test =pd.read_csv("test.csv", engine="c")


In [5]:
print("The shape of training dataset {}".format(train.shape))
print("The shape of label_data given is{}".format(train_label.shape))
print("The shape of test dataset is {}".format(test.shape))

The shape of training dataset (9999, 146)
The shape of label_data given is(9999, 34)
The shape of test dataset is (1999, 146)


In [6]:

#### Data Wrangling and Feature encoding ###

# initializing empty list


vec = DictVectorizer()
names_categorical = []



In [7]:
# seems like test file don't have column name
column_list = []
for column in train.columns:
    column_list.append(column)

# Add column names to test file
test.columns = column_list



In [8]:
test

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,x101,x102,x103,x104,x105,x106,x107,x108,x109,x110,x111,x112,x113,x114,x115,x116,x117,x118,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128,x129,x130,x131,x132,x133,x134,x135,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145
0,1698002,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.049834,0.945938,0.317427,0.482021,YES,NO,NO,NO,NO,3,0.895500,6,7,0.950468,0.0,0.88,4672,3311,YES,NO,YES,1,0.945032,0.471747,NO,NO,NO,NO,cr+kkNnNFV9YL0vz029hk3ohIDmGuABRVNhFe0ePZyo=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,0.832679,0.049834,0.945938,0.317427,0.480308,NO,NO,YES,NO,NO,4,0.804893,3,7,0.966777,0.0,0.84,4672,3311,NO,NO,NO,3,0.838115,0.471318,6Wv/YGbS0KDFv2UrATvlJcVtCjJOgnVQzWuuF5Ltv2k=,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.049834,0.945938,0.317427,0.482021,YES,NO,NO,NO,NO,1,0.725763,6,7,0.950468,0.0,0.88,4672,3311,YES,NO,YES,0,0.775294,0.471747,dBSc/QZM58O6miC4ULLhY0C4S6WIZLwy2oERlRo7Iaw=,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.049834,0.945938,0.317427,0.482021,YES,NO,NO,NO,NO,1,0.725763,6,7,0.950468,0.0,0.88,4672,3311,YES,NO,YES,0,0.775294,0.471747,0.832679,0.049834,0.945938,0.317427,0.482021,YES,NO,NO,NO,NO,0,0.725763,17,7,0.876992,1.0,0.866667,4672,3311,NO,NO,NO,5,0.945032,0.471318
1,1698003,NO,NO,MeBJ/ZzEIXfNKat4w1oeDxiMNKrAeY0PH41i00hpYDo=,tnLDGLnpYhzsik5+X+WPo4KQJoQA0TfWRlmEtQ3XNJQ=,1.415919,0.000000,1.000000,0.000000,0.703088,NO,NO,YES,YES,YES,4,0.536996,2,10,0.979821,0.0,1.00,1263,892,NO,NO,NO,8,0.557175,0.693587,NO,NO,NO,NO,W4WKBYyqy7COFohWfUyg2xuUPldk8b4hPsDCHjbePaU=,ABhwE8nsQMtFix2MDemgmGfoV68Tn3hLpZRXWVTQ0TM=,1.415919,0.000000,1.000000,0.000000,0.703088,NO,NO,NO,NO,NO,3,0.466368,6,10,0.939462,0.0,1.00,1263,892,NO,NO,NO,7,0.526906,0.693587,0+HOWI7C643OXIZ7LdoeNFzA7A0MkSH//diziszbII8=,NO,NO,W4WKBYyqy7COFohWfUyg2xuUPldk8b4hPsDCHjbePaU=,ABhwE8nsQMtFix2MDemgmGfoV68Tn3hLpZRXWVTQ0TM=,1.415919,0.000000,1.000000,0.000000,0.703088,NO,NO,NO,NO,NO,3,0.466368,6,10,0.939462,0.0,1.00,1263,892,NO,NO,NO,7,0.526906,0.693587,eI6XNOFa+S9J5wTICUEas3M6YE4TJ6rSTCo2NvqM16s=,NO,NO,W4WKBYyqy7COFohWfUyg2xuUPldk8b4hPsDCHjbePaU=,ABhwE8nsQMtFix2MDemgmGfoV68Tn3hLpZRXWVTQ0TM=,1.415919,0.000000,1.000000,0.000000,0.703088,NO,NO,NO,NO,NO,3,0.466368,6,10,0.939462,0.0,1.00,1263,892,NO,NO,NO,7,0.526906,0.693587,1.415919,0.000000,1.000000,0.000000,0.703088,NO,NO,YES,YES,YES,0,0.466368,9,10,0.919283,-1.0,1.000000,1263,892,NO,NO,NO,8,0.557175,0.693587
2,1698004,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,0,0.000000,0,0,0.000000,0.0,0.00,0,0,NaN,NaN,NaN,0,0.000000,0.000000,NO,NO,NO,NO,+X/rq+4qeKy2UVQN5xLQHSbri0Vdd/mYdG7WH/Kn+Pg=,oo9tGpHvTredpg9JkHgYbZAuxcwtSpQxU5mA/zUbxY8=,0.646703,0.059891,0.921960,0.531760,0.386558,NO,NO,NO,NO,NO,2,0.771325,6,9,0.939504,0.0,0.88,4672,3306,NO,NO,NO,3,0.831821,0.377568,nxOTdokWdl63qtAt4sgkSvyREdGu+ht1Iw4pcCaMi64=,YES,NO,QuSoZ63+jzXv3Ux1Y2LB+aSj7QCdPbrup3JSBj6Dmx8=,+yhSY//Hpg7u0bSA7NYmcmRFgv3bF4Tw3BMHrBqaTtA=,0.646703,0.059891,0.921960,0.531760,0.427226,YES,NO,YES,NO,YES,3,0.833031,1,7,0.992136,0.0,0.93,4672,3306,YES,NO,YES,1,0.840895,0.419949,B+EJpnEbkYtLnwDQYN1dP1rcfnoCnxAjKLYwQZE07Ew=,NO,NO,JA7Y7rY1wqr46lhqu5scuWov+Xw4neBTlPOPObhG0V8=,PjdztVAv+7nhxs8/+uAT0IRL7OcEHhLwHZ0IDTVbqcs=,0.646703,0.059891,0.921960,0.531760,0.416096,NO,NO,NO,NO,NO,13,0.161525,25,3,0.749546,0.0,0.86,4672,3306,NO,NO,NO,1,0.411978,0.405394,0.646703,0.059891,0.921960,0.531760,0.413099,YES,NO,NO,NO,NO,2,0.823351,5,3,0.952813,0.0,0.870000,4672,3

In [9]:

# Encoding of dataset
from sklearn.feature_extraction import DictVectorizer

X_numerical = []
X_test_numerical = []

vec = DictVectorizer()

names_categorical = []

train_with_labels.replace('YES', 1, inplace=True)
train_with_labels.replace('NO', 0, inplace = True)
train_with_labels.replace('nan', np.NaN, inplace = True)

# Encoding for test Dataset
test.replace('YES', 1, inplace=True)
test.replace('NO', 0, inplace = True)
test.replace('nan', np.NaN, inplace = True)

# checking all the columns types in data set
#train_with_labels.columns.map(type)
#train_with_labels.columns.map(type)


In [10]:
train_with_labels.columns =train_with_labels.columns.astype(str)
test.columns = test.columns.astype(str)


In [11]:
numeric_train_data = train.select_dtypes(include=[np.number])
categorical_train_data = train.select_dtypes(exclude=[np.number])

numeric_test_data = test.select_dtypes(include=[np.number])
categorical_test_data = test.select_dtypes(exclude=[np.number])


In [12]:
print(numeric_train_data.shape)
print(numeric_test_data.shape)
print(test.shape)

(9999, 86)
(1999, 136)
(1999, 146)


In [13]:
numeric_test_data.shape


(1999, 136)

- Make copy of training set
- Make copy of label data


In [14]:
X = numeric_train_data.copy()
y = train_label.copy()

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [16]:
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.multiclass import OneVsRestClassifier

In [17]:
sgd = SGDClassifier()
lr = LogisticRegression(solver='lbfgs')
svc = LinearSVC()

rfc = RandomForestClassifier(n_estimators = 10, n_jobs = 1)

In [18]:
def j_score(y_true, y_pred):
    jaccard = np.minimum(y_true, y_pred).sum(axis=1)/np.maximum(y_true, y_pred).sum(axis = 1)
    return jaccard.mean()*100

def print_score(y_pred, clf):
    print("clf:", clf.__class__.__name__)
    print('jaccard score: {}'.format(j_score(y_test, y_pred)))
    print("---------------------")


In [20]:
'''
for classifier in [sgd, lr, svc, rfc]:
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print_score(y_pred, classifier)
    
'''

'\nfor classifier in [sgd, lr, svc, rfc]:\n    clf = OneVsRestClassifier(classifier)\n    clf.fit(X_train, y_train)\n    y_pred = clf.predict(X_test)\n    print_score(y_pred, classifier)\n    \n'

In [22]:
# Different approch with victorizer
for name in train_with_labels.columns :    
    if name.startswith('x') :
        column_type, _ = max(Counter(map(lambda x: str(type(x)), train_with_labels[name])).items(), key = lambda x: x[1])
        
        # LOL expression
        if column_type == str(str) :
            train_with_labels[name] = map(str, train_with_labels[name])
            test[name] = map(str, test[name])

            names_categorical.append(name)
            #print (name, len(np.unique(train_with_labels[name])))
        else :
            X_numerical.append(train_with_labels[name].fillna(-999))
            X_test_numerical.append(test[name].fillna(-999))
        
X_numerical = np.column_stack(X_numerical)
X_test_numerical = np.column_stack(X_test_numerical)

X_sparse = vec.fit_transform(train_with_labels[names_categorical].T.to_dict().values())
X_test_sparse = vec.transform(test[names_categorical].T.to_dict().values())

print (X_numerical.shape, X_sparse.shape, X_test_numerical.shape, X_test_sparse.shape)

(9999, 135) (9999, 10) (1999, 135) (1999, 10)


In [23]:
X_numerical = np.nan_to_num(X_numerical)
X_test_numerical = np.nan_to_num(X_test_numerical)

### Build two level classifier, first train base level

In [24]:
from sklearn.metrics import roc_auc_score, f1_score, log_loss, make_scorer
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier

log_loss_scorer = make_scorer(log_loss, needs_proba = True)

y_columns = [name for name in train_with_labels.columns if name.startswith('y')]

X_numerical_base, X_numerical_meta, X_sparse_base, X_sparse_meta, y_base, y_meta = train_test_split(
    X_numerical, 
    X_sparse, 
    train_with_labels[y_columns].values,
    test_size = 0.5
)

X_meta = [] 
X_test_meta = []

print ("Build meta")

for i in range(y_base.shape[1]) :
    print (i)
    
    y = y_base[:, i]
    if len(np.unique(y)) == 2 : 
        rf = RandomForestClassifier(n_estimators = 10, n_jobs = 1)
        rf.fit(X_numerical_base, y)
        X_meta.append(rf.predict_proba(X_numerical_meta))
        X_test_meta.append(rf.predict_proba(X_test_numerical))

        svm = LinearSVC()
        svm.fit(X_sparse_base, y)
        X_meta.append(svm.decision_function(X_sparse_meta))
        X_test_meta.append(svm.decision_function(X_test_sparse))
        
X_meta = np.column_stack(X_meta)
X_test_meta = np.column_stack(X_test_meta)

Build meta
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [25]:
print (X_meta.shape, X_test_meta.shape)

(5000, 93) (1999, 93)


In [26]:
p_test = []

for i in range(y_base.shape[1]) :
    y = y_meta[:, i]

    constant = Counter(y)
    constant = constant[0] < 4 or constant[1] < 4
    
    predicted = None
    
    if constant :
        # Best constant
        constant_pred = np.mean(list(y_base[:, i]) + list(y_meta[:, i]))
        
        predicted = np.ones(X_test_meta.shape[0]) * constant_pred
        print ("%d is constant like: %f" % (i, constant_pred))
    else :
        rf = RandomForestClassifier(n_estimators=30, n_jobs = 1)
        rf.fit(np.hstack([X_meta, X_numerical_meta]), y)

        predicted = rf.predict_proba(np.hstack([X_test_meta, X_test_numerical]))

        predicted = predicted[:, 1]
        
        rf = RandomForestClassifier(n_estimators=30, n_jobs = 1)
        scores = cross_val_score(rf, np.hstack([X_meta, X_numerical_meta]), y, cv = 4, n_jobs = 1, scoring = log_loss_scorer)

        print (i, 'RF log-loss: %.4f ± %.4f, mean = %.6f' %(np.mean(scores), np.std(scores), np.mean(predicted)))

    
    p_test.append(
        predicted
    )
    
p_test = np.column_stack(p_test)

0 RF log-loss: 0.0664 ± 0.0202, mean = 0.008187
1 RF log-loss: 0.0160 ± 0.0123, mean = 0.000817
2 RF log-loss: 0.0547 ± 0.0305, mean = 0.026363
3 RF log-loss: 0.0197 ± 0.0133, mean = 0.015074
4 is constant like: 0.000200
5 RF log-loss: 0.1517 ± 0.0314, mean = 0.072168
6 RF log-loss: 0.1406 ± 0.0291, mean = 0.037052
7 is constant like: 0.000600
8 RF log-loss: 0.1077 ± 0.0131, mean = 0.076422
9 RF log-loss: 0.0964 ± 0.0589, mean = 0.017191
10 RF log-loss: 0.0163 ± 0.0122, mean = 0.001401
11 RF log-loss: 0.1578 ± 0.0256, mean = 0.068024
12 RF log-loss: 0.0786 ± 0.0337, mean = 0.014741
13 is constant like: 0.000000
14 RF log-loss: 0.0144 ± 0.0121, mean = 0.002335
15 RF log-loss: 0.0602 ± 0.0407, mean = 0.012706
16 is constant like: 0.000100
17 is constant like: 0.000100
18 is constant like: 0.000400
19 RF log-loss: 0.0121 ± 0.0113, mean = 0.003502
20 RF log-loss: 0.0317 ± 0.0121, mean = 0.008054
21 RF log-loss: 0.0528 ± 0.0122, mean = 0.004502
22 RF log-loss: 0.0096 ± 0.0119, mean = 0.0012

In [28]:
p_test.shape

(1999, 33)

In [29]:
d = pd.DataFrame(p_test,index=p_test[:,0])

In [30]:
d

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
0.000000,0.000000,0.0,0.000000,0.000000,0.0002,0.000000,0.000000,0.0006,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0001,0.0001,0.0004,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.033333,0.000000,0.933333
0.000000,0.000000,0.0,0.000000,0.000000,0.0002,0.000000,0.033333,0.0006,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0001,0.0001,0.0004,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.966667
0.000000,0.000000,0.0,0.000000,0.000000,0.0002,0.800000,0.000000,0.0006,0.000000,0.0,0.000000,0.800000,0.0,0.0,0.0,0.000000,0.0001,0.0001,0.0004,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.133333,0.133333,0.000000,0.066667
0.000000,0.000000,0.0,0.000000,0.000000,0.0002,0.000000,0.000000,0.0006,0.700000,0.0,0.033333,0.000000,0.0,0.0,0.0,0.000000,0.0001,0.0001,0.0004,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.266667,0.000000,0.033333,0.000000,0.033333
0.000000,0.000000,0.0,0.000000,0.000000,0.0002,0.000000,0.000000,0.0006,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0001,0.0001,0.0004,0.0,0.933333,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.033333,0.033333,0.0,0.000000,0.000000,0.0002,0.000000,0.033333,0.0006,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0001,0.0001,0.0004,0.0,0.300000,0.0,0.0,0.033333,0.000000,0.000000,0.3,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333
0.000000,0.000000,0.0,0.000000,0.000000,0.0002,0.000000,0.033333,0.0006,0.366667,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0001,0.0001,0.0004,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.166667,0.000000,0.000000,0.033333,0.466667
0.000000,0.000000,0.0,0.000000,0.000000,0.0002,0.000000,0.000000,0.0006,0.000000,0.0,0.000000,0.033333,0.0,0.0,0.0,0.000000,0.0001,0.0001,0.0004,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.066667,0.1,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
0.000000,0.000000,0.0,0.033333,0.000000,0.0002,0.000000,0.033333,0.0006,0.100000,0.0,0.000000,0.033333,0.0,0.0,0.0,0.000000,0.0001,0.0001,0.0004,0.0,0.000000,0.0,0.0,0.200000,0.033333,0.166667,0.0,0.033333,0.033333,0.000000,0.000000,0.000000,0.533333


In [31]:
y_pred= np.column_stack(d)


In [32]:
y_pred

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
        32]])